In [1]:
!pip install -q torch transformers huggingface_hub datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


In [2]:
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from getpass import getpass
# api_token = getpass("Enter your Hugging Face API token: ")


Enter your Hugging Face API token: ··········


In [ ]:
# import os
# os.environ["HF_HOME"] = os.path.expanduser("https://huggingface.co/nadika/nepali_complaints_classification")
# os.environ["HF_HOME"] = api_token


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,TrainingArguments,Trainer

In [6]:
import torch
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
from datasets import load_dataset,DatasetDict
from huggingface_hub import Repository

In [8]:
tokenizer=AutoTokenizer.from_pretrained('google/muril-base-cased')
model=AutoModelForSequenceClassification.from_pretrained('google/muril-base-cased')

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
dataset_file_path='/content/drive/MyDrive/major/cleaned_data_final.xlsx'

In [10]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
df = pd.read_excel(dataset_file_path)

In [12]:
df['गुनासो वर्ग'].unique()

array(['अर्थ सबन्धी', 'अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी',
       'कर्मचारी सम्वन्धी', 'खानेपानी', 'निर्माण कार्य सम्बन्धी',
       'पार्किङ्ग तथा मेट्रो पुलिस', 'प्राकृतिक श्रोत/साधन सम्बन्धी',
       'फोहोरमैला व्यवस्थापन', 'लागु पदार्थ सम्बन्धी',
       'वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी', 'शान्ति सुरक्षा',
       'सूचना तथा संचार', 'सोधपुछ, सुझाव, प्रशंसा सम्बन्धी',
       'स्वास्थ्यसँग सम्बन्धी'], dtype=object)

In [13]:
df.head()

,गुनासो वर्ग,गुनासो
0,अर्थ सबन्धी,मीनभवन र तिनकुनेको बीचमा रहेको एलजी शोरुम माथि...
1,अर्थ सबन्धी,- भ्याट बिल जारी गरिएको छैन
2,अर्थ सबन्धी,. जानकारी - अनुरोध\nकुनै पनि मानिसको जागिर अवध...
3,अर्थ सबन्धी,एटीएम कार्ड हराएको लगभग २ वर्ष जति भइसकेको छ न...
4,अर्थ सबन्धी,"वास्तवमा, यो उल्लिखित समस्या स्टक एक्सचेन्जसँग..."


In [14]:
df['गुनासो वर्ग'].value_counts()

वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी      2849
सोधपुछ, सुझाव, प्रशंसा सम्बन्धी             1883
पार्किङ्ग तथा मेट्रो पुलिस                  1701
कर्मचारी सम्वन्धी                           1337
अर्थ सबन्धी                                  860
खानेपानी                                     800
स्वास्थ्यसँग सम्बन्धी                        694
सूचना तथा संचार                              640
शान्ति सुरक्षा                               634
फोहोरमैला व्यवस्थापन                         630
निर्माण कार्य सम्बन्धी                       605
प्राकृतिक श्रोत/साधन सम्बन्धी                200
लागु पदार्थ सम्बन्धी                         157
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी      71
Name: गुनासो वर्ग, dtype: int64

In [15]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.2 MB/s eta 0:00:00


In [16]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.flow as naf
from nlpaug.util import Action
import random

In [17]:
counts=df['गुनासो वर्ग'].value_counts()
max_count= max(counts)
max_count

2849

In [18]:
# # Define the simple text augmentation function again
# def simple_augment_nepali_text(text, operation="insertion", changes=1):
#     words = text.split()

#     if operation == "insertion":
#         for _ in range(changes):
#             insert_position = random.randint(0, len(words) - 1)
#             words.insert(insert_position, "नेपाली")
#     elif operation == "deletion":
#         for _ in range(min(changes, len(words))):
#             delete_position = random.randint(0, len(words) - 1)
#             del words[delete_position]

#     return ' '.join(words)

In [19]:
# import pandas as pd
# import random

# # Assuming df is the DataFrame with your data
# # Replace 'category_column' with the actual name of your category column, e.g., 'गुनासो वर्ग'
# category_column = 'गुनासो वर्ग'

# counts = df[category_column].value_counts()
# max_count = max(counts)

# df_final = pd.DataFrame()

# for category, count in counts.items():
#     df_cat = df[df[category_column] == category]
#     augmentation_needed = max_count - count

#     # Check if augmentation is needed
#     if augmentation_needed > 0:
#         augmented_texts = []
#         for _ in range(augmentation_needed):
#             # Randomly select a text to augment
#             text_to_augment = df_cat.sample(n=1, replace=True)['गुनासो'].iloc[0]
#             # Apply the augmentation function
#             augmented_text = simple_augment_nepali_text(text_to_augment, operation="insertion", changes=2)
#             augmented_texts.append({'गुनासो': augmented_text, category_column: category})

#         # Create a DataFrame from the augmented texts
#         augmented_df = pd.DataFrame(augmented_texts)
#         # Combine the original and augmented data for this category
#         df_cat_final = pd.concat([df_cat, augmented_df])
#     else:
#         df_cat_final = df_cat

#     # Combine with the final DataFrame
#     df_final = pd.concat([df_final, df_cat_final], ignore_index=True)

# # Ensure the final DataFrame doesn't exceed the max_count for any category
# # This is a safeguard and may not be necessary if the above loop works correctly
# df_final_counts = df_final[category_column].value_counts()
# print(df_final_counts)


In [20]:
df_final=pd.DataFrame()
for category,count in counts.items():
  df_cat=df[df['गुनासो वर्ग']==category]
  if count < max_count:
    df_cat_oversampled=df_cat.sample(max_count,replace=True)
    df_cat_oversampled=df_cat_oversampled.reset_index(drop=True)
  else:
    df_cat_oversampled=df_cat

  df_final=pd.concat([df_final,df_cat_oversampled])


In [21]:
df_final['गुनासो वर्ग'].value_counts()

वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी      2849
सोधपुछ, सुझाव, प्रशंसा सम्बन्धी             2849
पार्किङ्ग तथा मेट्रो पुलिस                  2849
कर्मचारी सम्वन्धी                           2849
अर्थ सबन्धी                                 2849
खानेपानी                                    2849
स्वास्थ्यसँग सम्बन्धी                       2849
सूचना तथा संचार                             2849
शान्ति सुरक्षा                              2849
फोहोरमैला व्यवस्थापन                        2849
निर्माण कार्य सम्बन्धी                      2849
प्राकृतिक श्रोत/साधन सम्बन्धी               2849
लागु पदार्थ सम्बन्धी                        2849
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी    2849
Name: गुनासो वर्ग, dtype: int64

In [22]:
df_final.sample(5)

,गुनासो वर्ग,गुनासो
2629,खानेपानी,"हाम्रो घरमा पानी नआएको धेरै भयो, पानी कार्यलयल..."
986,प्राकृतिक श्रोत/साधन सम्बन्धी,नेपाल सरकारले तोकेको iee मापदणड बिपरित नदि जन्...
1752,कर्मचारी सम्वन्धी,"श्रीमान् सूचना अधिकारी ज्यू, \r\nहेलो सरकार\ न..."
1700,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,इलाका प्रशासन मंगलबारे मा पासपोर्ट वनाउन जादा ...
1119,"सोधपुछ, सुझाव, प्रशंसा सम्बन्धी","नेपाल सरकार संग अनुरोध छ, विदेश जाने नागरिक ले..."


In [23]:
# Data Preprocessing
nepali_stopwords = set(stopwords.words('nepali'))
english_stopwords = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[#\\/।(),०-९<<?!,—–’‘:\u200d]', '', text)
    text = text.strip('"')

    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in nepali_stopwords and word.lower() not in english_stopwords]
    processed_text = ' '.join(filtered_words)
    return processed_text



In [24]:
df_final['गुनासो'] = df_final['गुनासो'].apply(preprocess_text)

In [25]:

df_final['गुनासो वर्ग'].value_counts()

वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी      2849
सोधपुछ, सुझाव, प्रशंसा सम्बन्धी             2849
पार्किङ्ग तथा मेट्रो पुलिस                  2849
कर्मचारी सम्वन्धी                           2849
अर्थ सबन्धी                                 2849
खानेपानी                                    2849
स्वास्थ्यसँग सम्बन्धी                       2849
सूचना तथा संचार                             2849
शान्ति सुरक्षा                              2849
फोहोरमैला व्यवस्थापन                        2849
निर्माण कार्य सम्बन्धी                      2849
प्राकृतिक श्रोत/साधन सम्बन्धी               2849
लागु पदार्थ सम्बन्धी                        2849
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी    2849
Name: गुनासो वर्ग, dtype: int64

In [26]:
label_encoder = LabelEncoder()
df_final['encoded_labels'] = label_encoder.fit_transform(df_final['गुनासो वर्ग'])

In [27]:
df_final.head()

,गुनासो वर्ग,गुनासो,encoded_labels
6361,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अछाम जिल्ला रेकर्ड भन्छ,9
6362,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अझ यसमा लाईसेन्सको थप्न पाए हुन्थ्यो,9
6363,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अपडेट नभएको सर नागरिक apps माथी उल्लेखित न.हान...,9
6364,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अञ्चल लेख्न लेख्नी बाघ मति मात्रै आउनुपर्ने,9
6365,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अधिकृत छैठौको भोली फम भर्ने अन्तिम मिति हिजो क...,9


In [28]:
df_final.tail()

,गुनासो वर्ग,गुनासो,encoded_labels
2844,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,मकवानपुरका एसपी सोमेन्द्रसिंह राठौरले हवल्दार ...,1
2845,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,कर चुक्ता लिन जादा प्रती कर चुक्ता 2000 नभइ कर...,1
2846,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,कपिलवस्तु पिपरा चेकपोस्टमा ट्राफिक प्रहरी जवान...,1
2847,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,डडेल्धुरा अस्पताल डडेल्धुरामा कार्यरत हिरा कुम...,1
2848,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,पाँचथर जिल्ला अस्पतालमा कर्मचारी राख्दा अनियमि...,1


In [29]:
encodings = tokenizer(df_final['गुनासो'].tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")

In [30]:
train_indices, val_indices = train_test_split(np.arange(len(df_final)), test_size=0.1, stratify=df_final['encoded_labels'])


In [31]:
# Custom dataset class
class NepaliComplaintsDataset(Dataset):
    def __init__(self, encodings, labels, indices):
        self.encodings = {key: val[indices] for key, val in encodings.items()}
        self.labels = labels[indices]

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [32]:

# Creating datasets
train_dataset = NepaliComplaintsDataset(encodings, np.array(df_final['encoded_labels']), train_indices)
val_dataset = NepaliComplaintsDataset(encodings, np.array(df_final['encoded_labels']), val_indices)

In [33]:
# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [34]:
num_labels = len(np.unique(df_final['encoded_labels']))


In [35]:
model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



In [37]:
!huggingface-cli whoami

nadika


In [38]:
# hub_model_id = "nadika/nepali_complaints_classification"

In [39]:
training_args = TrainingArguments(
    output_dir='nepali_complaints_classification_muril',          # Directory for saving model and outputs
    num_train_epochs=4,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=1000,                # Number of warmup steps
    weight_decay=0.05,               # Weight decay rate #l2 regularization AdamW
    logging_dir='./logs',            # Directory for logging
    logging_steps=10,                # Log every X updates steps
    evaluation_strategy="epoch",     # Evaluate at the end of every epoch
    save_strategy="epoch",           # Save model at the end of every epoch
    learning_rate=2e-5,
    load_best_model_at_end=True,     # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Metric to optimize over
    push_to_hub=True,
    # hub_model_id=hub_model_id,
)

In [40]:
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }


In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    # compute_metrics=compute_metrics
)

In [42]:
trainer.train()

<ipython-input-31-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.916000,0.710256
2,0.272500,0.320635
3,0.167800,0.268524
4,0.204000,0.242388


<ipython-input-31-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-31-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-31-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=8976, training_loss=0.6452044372259242, metrics={'train_runtime': 4413.1078, 'train_samples_per_second': 32.537, 'train_steps_per_second': 2.034, 'total_flos': 3.778366070938829e+16, 'train_loss': 0.6452044372259242, 'epoch': 4.0})

In [43]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Ashwini1412/nepali_complaints_classification_muril/commit/f7630e6ffb145498e24bb313b71bf14470acede8', commit_message='End of training', commit_description='', oid='f7630e6ffb145498e24bb313b71bf14470acede8', pr_url=None, pr_revision=None, pr_num=None)

In [44]:
trainer.evaluate()

<ipython-input-31-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.24238833785057068,
 'eval_runtime': 37.5769,
 'eval_samples_per_second': 106.156,
 'eval_steps_per_second': 6.653,
 'epoch': 4.0}

In [67]:
tokenizer.push_to_hub("nadika/nepali_complaints_classification_muril")

README.md:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ashwini1412/nepali_complaints_classification_muril/commit/e40d2d13436e75031d1cbab15fa368d7df2ea4e3', commit_message='Upload tokenizer', commit_description='', oid='e40d2d13436e75031d1cbab15fa368d7df2ea4e3', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
labels = label_encoder.classes_  # This gets the ordered list of class names
id2label = dict(enumerate(labels))
id2label

{0: 'अर्थ सबन्धी',
 1: 'अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी',
 2: 'कर्मचारी सम्वन्धी',
 3: 'खानेपानी',
 4: 'निर्माण कार्य सम्बन्धी',
 5: 'पार्किङ्ग तथा मेट्रो पुलिस',
 6: 'प्राकृतिक श्रोत/साधन सम्बन्धी',
 7: 'फोहोरमैला व्यवस्थापन',
 8: 'लागु पदार्थ सम्बन्धी',
 9: 'वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी',
 10: 'शान्ति सुरक्षा',
 11: 'सूचना तथा संचार',
 12: 'सोधपुछ, सुझाव, प्रशंसा सम्बन्धी',
 13: 'स्वास्थ्यसँग सम्बन्धी'}

In [47]:
label2id = {label: id for id, label in id2label.items()}
label2id

{'अर्थ सबन्धी': 0,
 'अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी': 1,
 'कर्मचारी सम्वन्धी': 2,
 'खानेपानी': 3,
 'निर्माण कार्य सम्बन्धी': 4,
 'पार्किङ्ग तथा मेट्रो पुलिस': 5,
 'प्राकृतिक श्रोत/साधन सम्बन्धी': 6,
 'फोहोरमैला व्यवस्थापन': 7,
 'लागु पदार्थ सम्बन्धी': 8,
 'वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी': 9,
 'शान्ति सुरक्षा': 10,
 'सूचना तथा संचार': 11,
 'सोधपुछ, सुझाव, प्रशंसा सम्बन्धी': 12,
 'स्वास्थ्यसँग सम्बन्धी': 13}

In [45]:
model.save_pretrained('./my_muril_model')
tokenizer.save_pretrained('./my_muril_model')

('./my_muril_model/tokenizer_config.json',
 './my_muril_model/special_tokens_map.json',
 './my_muril_model/vocab.txt',
 './my_muril_model/added_tokens.json',
 './my_muril_model/tokenizer.json')

In [68]:
model_name="./my_muril_model"

In [69]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_name)


In [70]:
config.id2label = id2label
config.label2id = label2id

In [71]:
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [72]:
config.save_pretrained(model_name)

In [73]:
model_name="./my_muril_model"
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [74]:
repo_name="Ashwini1412/nepali_complaints_classification_muril"

In [75]:
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ashwini1412/nepali_complaints_classification_muril/commit/15fa958f27c1c82e368496d0162f2b09b4491fba', commit_message='Upload tokenizer', commit_description='', oid='15fa958f27c1c82e368496d0162f2b09b4491fba', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# model_id = "nadika/nepali_complaints_classification"  # replace with your model's repository name
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSequenceClassification.from_pretrained(model_id)

In [55]:


def preprocess_and_tokenize(text):
    # Assuming the `preprocess_text` is your text cleaning and preprocessing function
    # Make sure to include the preprocess_text function or adjust this part accordingly
    processed_text = preprocess_text(text)  # Use your preprocessing here
    encoded_input = tokenizer(processed_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    return encoded_input

def predict(text):
    # Preprocess and tokenize the text
    encoded_input = preprocess_and_tokenize(text)

    # Generate predictions
    with torch.no_grad():
        output = model(**encoded_input)

    # The output is logits; apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(output.logits, dim=-1)

    # Convert probabilities to numpy for easier handling
    probabilities = probabilities.detach().cpu().numpy()

    # Get the most likely class index
    predicted_class_idx = probabilities.argmax()

    # Decode the predicted class index to the class name using the model's config
    predicted_class = model.config.id2label[predicted_class_idx]
    return predicted_class, probabilities[0][predicted_class_idx]




In [56]:
# Example text for prediction
new_text = "यस महानगरपालिकाको सुनाकोठी नखिपोट नख्खुडोल गोकुल आवास कान्तिपुर कोलोनीका वासिन्दाले दिनरात यो धुंवाको सास कहिलेसम्म फेर्नुपर्ने हो हजुर?"
predicted_class, probability = predict(new_text)
print(f"Predicted Class: {predicted_class}, Probability: {probability}")

Predicted Class: सोधपुछ, सुझाव, प्रशंसा सम्बन्धी, Probability: 0.709530770778656


In [57]:
# Example text for prediction
new_text = "सुनसरी जिल्ला इटहरी उप महानगर भित्र अवस्थित धरान पुग्ने यो बाटो । कुन ठेकेदारले यो काम गर्दा कति पैसा कुम्ल्याउन पाइने हो र आफ्नो निर्वाचन ताका खर्च भएको करोड उठाउन पाइन्छ भनेर त हैन ?"
predicted_class, probability = predict(new_text)
print(f"Predicted Class: {predicted_class}, Probability: {probability}")

Predicted Class: सोधपुछ, सुझाव, प्रशंसा सम्बन्धी, Probability: 0.9265208840370178


In [58]:
# Example text for prediction
new_text = "इटहरीदेखि पूर्वाञ्चल विश्वविद्यालय, माेरङ हुँदै पूर्व जाने बाटाे जुन लालभित्तीमा गएर टुङ्गिन्छ, पिच गर्नुपर्‍याे। बीचमा दुइटा ठूला खाेला छन्, त्यसमा पुल हाल्नुपर्‍याे। याे बाटाेलाई वैकल्पिक राजमार्गकाे रुपमा विकास गर्नुप्रयो।"
predicted_class, probability = predict(new_text)
print(f"Predicted Class: {predicted_class}, Probability: {probability}")

Predicted Class: निर्माण कार्य सम्बन्धी, Probability: 0.9899699091911316


In [59]:
# Example text for prediction
new_text = "मेयरले उपभोक्ता समितिसँग ३० प्रतिशत कमिसन लिएपछि विकासको काम गुणस्तरहीन, दबाब झेल्न नसकेर प्रशासकीय अधिकृतको भागाभाग। बाह्रबिसेका मेयरको मनोमानी- दोहोरीमा रमाइलो गरेको बिलसमेत नगरपालिकाबाटै भुक्तानी गर्न दबाब मेयरले उपभोक्ता समितिसँग ३० प्रतिशत कमिसन लिएपछि विकासको काम गुणस्तरहीन, दबाब झेल्न नसकेर प्रशासकीय अधिकृतको भागाभाग।"
predicted_class, probability = predict(new_text)
print(f"Predicted Class: {predicted_class}, Probability: {probability}")

Predicted Class: कर्मचारी सम्वन्धी, Probability: 0.7910566329956055


In [60]:
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [62]:
predictions = []
labels = []

with torch.no_grad():
    for batch in val_loader:
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        model.to(device)
        output = model(inputs, attention_mask=attention_mask)
        logits = output.logits.to(device)
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        predictions.extend(preds)
        labels.extend(batch['labels'].cpu().numpy())

<ipython-input-31-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [63]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Calculate accuracy
accuracy = accuracy_score(labels, predictions)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, and F1 score
precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
print(f"Precision: {precision}\nRecall: {recall}\nF1 Score: {f1}")


Accuracy: 0.9450990223113562
Precision: 0.9453147910161994
Recall: 0.9450990223113562
F1 Score: 0.9450354734260795


In [66]:
import shutil
from google.colab import files

# Directory containing the saved model and tokenizer
directory_path = './my_muril_model'

# Zip the directory
shutil.make_archive('my_muril_model', 'zip', directory_path)

# Download the zip file
files.download('my_muril_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>